In [38]:
import numpy as np
import matplotlib.pyplot as plt
import qiskit as qk
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram
import random
from collections import Counter

from generator import generateCliffordCircuit

from runner import *

To implement the ACES protocol, our test circuits of concern are Clifford Circuits with one layer of single qubit gates (the "easy" layer) interleaved with a layer containing a single multi-qubit gate (the "hard" layer). The gate set that a user specifies will consist of the gates they want to characterize the error of. For our implementation we use {X,H,Z,I,S,CNOT} as our gate set. Below, we generate a random Clifford circuit with a layer of single qubit gates and a layer of multi-qubit gates. [https://arxiv.org/pdf/1009.3639, https://arxiv.org/pdf/1109.6887]

In [39]:
width = 5 # num_qubits
depth = 5 # number of layers in the circuit (easy + hard)
singleGateSet = ['X', 'H', 'Z', 'I', 'S']
doubleGateSet = ['CNOT_C', 'CNOT_T']
twirlingGateSet = ['X','Y' 'Z', 'I']

circuit = generateCliffordCircuit(width, depth, singleGateSet, doubleGateSet)
qiskitCircuit = transpileListToQiskitCircuit(circuit)
print(circuit)
if depth < 10:
    print(qiskitCircuit)

[['S', 'X', 'X', 'I', 'X'], ['I', 'I', 'I', 'CNOT_C', 'CNOT_T'], ['I', 'Z', 'Z', 'S', 'I'], ['CNOT_T', 'I', 'CNOT_C', 'I', 'I'], ['S', 'I', 'S', 'X', 'H']]
     ┌───┐ ░       ░ ┌───┐ ░ ┌───┐ ░ ┌───┐
q_0: ┤ S ├─░───────░─┤ I ├─░─┤ X ├─░─┤ S ├
     ├───┤ ░       ░ ├───┤ ░ └─┬─┘ ░ ├───┤
q_1: ┤ X ├─░───────░─┤ Z ├─░───┼───░─┤ I ├
     ├───┤ ░       ░ ├───┤ ░   │   ░ ├───┤
q_2: ┤ X ├─░───────░─┤ Z ├─░───■───░─┤ S ├
     ├───┤ ░       ░ ├───┤ ░       ░ ├───┤
q_3: ┤ I ├─░───■───░─┤ S ├─░───────░─┤ X ├
     ├───┤ ░ ┌─┴─┐ ░ ├───┤ ░       ░ ├───┤
q_4: ┤ X ├─░─┤ X ├─░─┤ I ├─░───────░─┤ H ├
     └───┘ ░ └───┘ ░ └───┘ ░       ░ └───┘


Next, we will G-twirl the circuit. This is the generalization of Pauli-twirling, which twirls every Clifford gate in the circuit instead of just the "hard" layers, and was first introduced in the ACES protocol. Below we G-twirl the circuit we just made. We first randomly choose a layer of Paulis to append before the current layer, then calculate The pauli $P' = G*P*G^{\dag}$ for each gate, $G$, in the layer.

In [40]:
G_twirling(qiskitCircuit).draw()

┌───────────────┐┌───┐┌───────────────┐ ░ ┌───────────────┐     »
q_0: ┤0              ├┤ S ├┤0              ├─░─┤0              ├─────»
     │               │├───┤│               │ ░ │               │     »
q_1: ┤1              ├┤ X ├┤1              ├─░─┤1              ├─────»
     │               │├───┤│               │ ░ │               │     »
q_2: ┤2 Pauli(ZYIZZ) ├┤ X ├┤2 Pauli(ZYIZZ) ├─░─┤2 Pauli(YYIXY) ├─────»
     │               │├───┤│               │ ░ │               │     »
q_3: ┤3              ├┤ I ├┤3              ├─░─┤3              ├──■──»
     │               │├───┤│               │ ░ │               │┌─┴─┐»
q_4: ┤4              ├┤ X ├┤4              ├─░─┤4              ├┤ X ├»
     └───────────────┘└───┘└───────────────┘ ░ └───────────────┘└───┘»
«     ┌─────────┐ ░ ┌───────────────┐┌───┐┌─────────┐ ░ ┌───────────────┐┌───┐»
«q_0: ┤0        ├─░─┤0              ├┤ I ├┤0        ├─░─┤0              ├┤ X ├»
«     │         │ ░ │               │├───┤│         │ ░ │               │└─┬─┘»
«q_1: ┤1        ├─░─┤1              ├┤ Z ├┤1        ├─░─┤1              ├──┼──»
«     │         │ ░ │               │├───┤│         │ ░ │               │  │  »
«q_2: ┤2 -ZXIXY ├─░─┤2 Pauli(YYIIX) ├┤ Z ├┤2 -YXIIX ├─░─┤2 Pauli(XIXXI) ├──■──»
«     │         │ ░ │               │├───┤│         │ ░ │               │     »
«q_3: ┤3        ├─░─┤3              ├┤ S ├┤3        ├─░─┤3              ├─────»
«     │         │ ░ │               │├───┤│         │ ░ │               │     »
«q_4: ┤4        ├─░─┤4              ├┤ I ├┤4        ├─░─┤4              ├─────»
«     └─────────┘ ░ └───────────────┘└───┘└─────────┘ ░ └───────────────┘     »
«     ┌───────────────┐ ░ ┌───────────────┐┌───┐┌─────────┐
«q_0: ┤0              ├─░─┤0              ├┤ S ├┤0        ├
«     │               │ ░ │               │├───┤│         │
«q_1: ┤1              ├─░─┤1              ├┤ I ├┤1        ├
«     │               │ ░ │               │├───┤│         │
«q_2: ┤2 Pauli(XIXXX) ├─░─┤2 Pauli(XIYIZ) ├┤ S ├┤2 -ZIXIZ ├
«     │               │ ░ │               │├───┤│         │
«q_3: ┤3              ├─░─┤3              ├┤ X ├┤3        ├
«     │               │ ░ │               │├───┤│         │
«q_4: ┤4              ├─░─┤4              ├┤ H ├┤4        ├
«     └───────────────┘ ░ └───────────────┘└───┘└─────────┘

We then make an ensemble of twirled circuits. Note, each circuit in this ensemble implements the same unitary as $PGP'=PG(GPG^{\dag})=PG(G^{\dag}PG)=G$

In [41]:
circuit_ensemble = [G_twirling(qiskitCircuit) for _ in range(10)]

To implement the measurement procedue on these circuits which will yield an estimator for the total circuit error, we create a prepartion circuit which prepares the input state in either the $|\psi+\rangle$ or $|\psi-\rangle$ state of the input Pauli, P. Then we compute $P'=GPG^{\dag}$ and build a measurement circuit which measures in the $|\psi'+\rangle$ or $|\psi'-\rangle$, the eigen basis of $P'$.

In [42]:
input_pauli = "YYIZX"
b,c = prep_circuit(input_pauli)
print("prep_circuit")
print(b)
a = measure_circuit(Clifford_Permute(qiskitCircuit,input_pauli))
print("measure_circuit")
print(a)

prep_circuit
     ┌───┐┌───┐┌───┐
q_0: ┤ X ├┤ H ├┤ S ├
     ├───┤├───┤├───┤
q_1: ┤ X ├┤ H ├┤ S ├
     └───┘└───┘└───┘
q_2: ───────────────
     ┌───┐          
q_3: ┤ X ├──────────
     ├───┤          
q_4: ┤ H ├──────────
     └───┘          
measure_circuit
     ┌─────┐┌───┐┌─┐   
q_0: ┤ Sdg ├┤ H ├┤M├───
     ├─────┤├───┤└╥┘┌─┐
q_1: ┤ Sdg ├┤ H ├─╫─┤M├
     └─────┘└───┘ ║ └╥┘
q_2: ─────────────╫──╫─
       ┌─┐        ║  ║ 
q_3: ──┤M├────────╫──╫─
       └╥┘   ┌─┐  ║  ║ 
q_4: ───╫────┤M├──╫──╫─
        ║    └╥┘  ║  ║ 
c: 4/═══╩═════╩═══╩══╩═
        2     3   0  1 


$\Lambda_{C,P}$ estimation proceeds by randomly sampling a circuit, $C$, from the ensemble, then randomly preparing either $|\psi+\rangle$ or $|\psi-\rangle$ and measuring in the $P'$ basis. After many successive iterations, we can build an unbiased estimator for the total circuit error, $\Lambda_{C,P}$,which is 
$$\Lambda_{C,P}^{est} = |\langle \psi'+|U_C|\psi+\rangle|^2 - |\langle \psi'-|U_C|\psi+\rangle|^2 - \langle \psi'+|U_C|\psi-\rangle|^2 + \langle \psi'-|U_C|\psi-\rangle|^2$$

For a circuit without error, we should see $\Lambda_{C,P}^{est} = \pm 1$, as we have $C(P_{in})=\Lambda_{C,P}P'$. We show below that our function properly estimates the total circuit error for a circuit with no error.

In [43]:
out = est_Lambda(input_pauli,[G_twirling(qiskitCircuit) for _ in range(10)],1000)
print(out)

1.0


In fact, our lambda estimator works as expected for all gates in our gate set.

In [44]:
d = {}
for inp in ["X","Y","Z"]:
    for g in singleGateSet:
        a = qk.QuantumCircuit(1)
        if g == "X":
            a.x(0)
        elif g == "Y":
            a.y(0)
        elif g == "Z":
            a.z(0)
        elif g == "H":
            a.h(0)
        elif g == "S":
            a.s(0)
        elif g == "I":
            a.id(0)
        key = ("Input Puali: " + str(inp),"Gate: "+str(g),"Output Pauli: " + str(Clifford_Permute(a,inp)))
        d[key] = est_Lambda(inp,[a],1000)
        print(key)
        print(d[key])

print(len(d))

('Input Puali: X', 'Gate: X', 'Output Pauli: X')
1.0
('Input Puali: X', 'Gate: H', 'Output Pauli: Z')
1.0
('Input Puali: X', 'Gate: Z', 'Output Pauli: -X')
-1.0
('Input Puali: X', 'Gate: I', 'Output Pauli: X')
1.0
('Input Puali: X', 'Gate: S', 'Output Pauli: Y')
1.0
('Input Puali: Y', 'Gate: X', 'Output Pauli: -Y')
-1.0
('Input Puali: Y', 'Gate: H', 'Output Pauli: -Y')
-1.0
('Input Puali: Y', 'Gate: Z', 'Output Pauli: -Y')
-1.0
('Input Puali: Y', 'Gate: I', 'Output Pauli: Y')
1.0


('Input Puali: Y', 'Gate: S', 'Output Pauli: -X')
-1.0
('Input Puali: Z', 'Gate: X', 'Output Pauli: -Z')
-1.0
('Input Puali: Z', 'Gate: H', 'Output Pauli: X')
1.0
('Input Puali: Z', 'Gate: Z', 'Output Pauli: Z')
1.0
('Input Puali: Z', 'Gate: I', 'Output Pauli: Z')
1.0
('Input Puali: Z', 'Gate: S', 'Output Pauli: Z')
1.0
15


In [45]:
import itertools

combinations = list(itertools.product(["X","Y","Z","I"], repeat=2))
combinations = [''.join(tuple_of_strings) for tuple_of_strings in combinations]
combinations.remove("II")

d = {}
for inp in combinations:
    for i in [0,1]:
        a = qk.QuantumCircuit(2)
        if i == 0:  
            a.cx(0,1)
            g = "CNOT(0,1)"
        elif i == 1:
            a.cx(1,0)
            g = "CNOT(1,0)"
        key = ("Input Puali: " + str(inp),"Gate: "+str(g),"Output Pauli: " + str(Clifford_Permute(a,inp)))
        d[key] = est_Lambda(inp,[a],1000)
        print(key)
        print(d[key])
print(len(d))

('Input Puali: XX', 'Gate: CNOT(0,1)', 'Output Pauli: IX')
1.0
('Input Puali: XX', 'Gate: CNOT(1,0)', 'Output Pauli: XI')
1.0
('Input Puali: XY', 'Gate: CNOT(0,1)', 'Output Pauli: ZY')
1.0


('Input Puali: XY', 'Gate: CNOT(1,0)', 'Output Pauli: YI')
1.0
('Input Puali: XZ', 'Gate: CNOT(0,1)', 'Output Pauli: -YY')
-1.0
('Input Puali: XZ', 'Gate: CNOT(1,0)', 'Output Pauli: ZX')
1.0
('Input Puali: XI', 'Gate: CNOT(0,1)', 'Output Pauli: XX')
1.0
('Input Puali: XI', 'Gate: CNOT(1,0)', 'Output Pauli: IX')
1.0
('Input Puali: YX', 'Gate: CNOT(0,1)', 'Output Pauli: IY')
1.0
('Input Puali: YX', 'Gate: CNOT(1,0)', 'Output Pauli: YZ')
1.0
('Input Puali: YY', 'Gate: CNOT(0,1)', 'Output Pauli: -ZX')
-1.0
('Input Puali: YY', 'Gate: CNOT(1,0)', 'Output Pauli: -XZ')
-1.0
('Input Puali: YZ', 'Gate: CNOT(0,1)', 'Output Pauli: YX')
1.0
('Input Puali: YZ', 'Gate: CNOT(1,0)', 'Output Pauli: IY')
1.0
('Input Puali: YI', 'Gate: CNOT(0,1)', 'Output Pauli: XY')
1.0
('Input Puali: YI', 'Gate: CNOT(1,0)', 'Output Pauli: ZY')
1.0
('Input Puali: ZX', 'Gate: CNOT(0,1)', 'Output Pauli: XZ')
1.0
('Input Puali: ZX', 'Gate: CNOT(1,0)', 'Output Pauli: -YY')
-1.0
('Input Puali: ZY', 'Gate: CNOT(0,1)', 'Output 

Then we introduce noise to our circuit.

In [46]:
# error probabilities
px = 0.05  # adjust the probability as needed
py = 0.10  # adjust the probability as needed
pz = 0.20  # adjust the probability as needed
qiskitCircuitNoisy = transpileListToQiskitCircuit(circuit, noise=True, px=px, py=py, pz=pz)
print(qiskitCircuitNoisy)
print(G_twirling_noisy(qiskitCircuitNoisy))

     ┌───┐ ░ ┌───┐ ░       ░       ░ ┌───┐ ░ ┌───┐ ░ ┌───┐ ░ ┌───┐ ░ ┌───┐ ░ »
q_0: ┤ S ├─░─┤ I ├─░───────░───────░─┤ I ├─░─┤ Y ├─░─┤ X ├─░─┤ I ├─░─┤ S ├─░─»
     ├───┤ ░ ├───┤ ░       ░       ░ ├───┤ ░ ├───┤ ░ └─┬─┘ ░ └───┘ ░ ├───┤ ░ »
q_1: ┤ X ├─░─┤ I ├─░───────░───────░─┤ Z ├─░─┤ I ├─░───┼───░───────░─┤ I ├─░─»
     ├───┤ ░ ├───┤ ░       ░       ░ ├───┤ ░ ├───┤ ░   │   ░ ┌───┐ ░ ├───┤ ░ »
q_2: ┤ X ├─░─┤ X ├─░───────░───────░─┤ Z ├─░─┤ I ├─░───■───░─┤ X ├─░─┤ S ├─░─»
     ├───┤ ░ ├───┤ ░       ░ ┌───┐ ░ ├───┤ ░ ├───┤ ░       ░ └───┘ ░ ├───┤ ░ »
q_3: ┤ I ├─░─┤ I ├─░───■───░─┤ I ├─░─┤ S ├─░─┤ Y ├─░───────░───────░─┤ X ├─░─»
     ├───┤ ░ ├───┤ ░ ┌─┴─┐ ░ ├───┤ ░ ├───┤ ░ ├───┤ ░       ░       ░ ├───┤ ░ »
q_4: ┤ X ├─░─┤ Z ├─░─┤ X ├─░─┤ Z ├─░─┤ I ├─░─┤ Z ├─░───────░───────░─┤ H ├─░─»
     └───┘ ░ └───┘ ░ └───┘ ░ └───┘ ░ └───┘ ░ └───┘ ░       ░       ░ └───┘ ░ »
«     ┌───┐
«q_0: ┤ I ├
«     ├───┤
«q_1: ┤ I ├
«     ├───┤
«q_2: ┤ Z ├
«     ├───┤
«q_3: ┤ I ├
«     ├───┤
«q_4: ┤ I ├
«     └───┘

You can see above an example of random noise being introduced after every gate.

In [47]:
circuit = generateCliffordCircuit(width, depth, singleGateSet, doubleGateSet)
qiskitCircuitNoisy = transpileListToQiskitCircuit(circuit, noise=True, px=px, py=py, pz=pz)
noisy_circuit_ensemble = [G_twirling_noisy(qiskitCircuitNoisy) for _ in range(10)]
print(est_Lambda(input_pauli,noisy_circuit_ensemble,1000))

-1.0


We can generate the A matrix and solve for the $\lambda$ values for each gate and calculate their corresponding error rates with the below code.

In [ ]:
# generate A matrix
A = generate_A(width=width, depth=depth)

Lam_vec = [est_Lambda(input_pauli,noisy_circuit_ensemble,1000) for row in A] # needs implementing

# return error vector
lambda_vector = Amatrixsolve(A, Lam_vec)
print("LAMBDA VECTOR: ", lambda_vector)
error_vector = error(lambda_vector)